## A Critical Look at Cuckoo Search

This lab explores the common issue (powszechny problem) of seemingly (pozornie) new optimization algorithms that may not offer genuinely (naprawdę) novel ideas. We'll first examine the original Cuckoo Search paper and then critically analyze it (krytycznie zanalizujemy) using the paper "An analysis of why cuckoo search does not bring any novel ideas to optimization." This exercise builds on our previous discussions of evolutionary algorithms like Differential Evolution and the  (pułapek) of poor research in fields like Neuroevolution, focusing here on the problem of redundant (powielania) concepts in optimization.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation #animacje
from matplotlib.collections import PathCollection #kolekcje ścieżek
from typing import Callable
from dataclasses import dataclass
from math import gamma #gamma - funkcja eulera

### Optimization Problems

This cell defines three common benchmark functions, Sphere, Rosenbrock, and Rastrigin, used to test optimization algorithms. We also used these functions earlier to evaluate Adam, Momentum, and CMA-ES.

In [7]:
def sphere(x: np.ndarray) -> float:
    return float(np.sum(x**2))


def rosenbrock(x: np.ndarray) -> float:
    return float(np.sum(100.0 * (x[1:] - x[:-1] ** 2.0) ** 2.0 + (1.0 - x[:-1]) ** 2.0))


def rastrigin(x: np.ndarray) -> float:
    A: float = 10.0
    return float(A * len(x) + np.sum(x**2 - A * np.cos(2 * np.pi * x)))

BOUNDS = [(-5, 5), (-5, 5)]

### Visualizing Search Dynamics

In [9]:
def animate_cs(
    func: Callable[[np.ndarray], float],
    history: list[np.ndarray],
    bounds: list[tuple[float, float]] = BOUNDS,
    frames: int | None = None,
    filename: str = "cs_animation.gif",
) -> None:
    """
    Creates and saves a GIF showing how the CS population moves over generations.
    """
    if frames is None:
        frames = len(history)

    assert len(bounds) == 2, "This function only supports 2D visualization (expected 2 bounds)."
    x_bounds = (bounds[0][0], bounds[0][1])
    y_bounds = (bounds[1][0], bounds[1][1])

    x = np.linspace(x_bounds[0], x_bounds[1], 200)
    y = np.linspace(y_bounds[0], y_bounds[1], 200)
    X, Y = np.meshgrid(x, y)
    coords = np.vstack([X.ravel(), Y.ravel()]).T
    Z = np.array([func(pt) for pt in coords]).reshape(X.shape)

    fig, ax = plt.subplots(figsize=(8, 6))
    contour = ax.contourf(X, Y, Z, levels=20, cmap="viridis")
    fig.colorbar(contour, ax=ax)
    
    def update(i: int):
        # Remove any existing scatter plots (PathCollection instances)
        for coll in ax.collections:
            if isinstance(coll, PathCollection):
                coll.remove()

        ax.set_title(f"Generation {i}")

        DECAY_CONSTANT = 10

        # Plot all generations up to i, with alpha decreasing for older points
        for j in range(i + 1):
            age = i - j
            alpha = float(np.exp(-age / DECAY_CONSTANT))
            pop_j = history[j]
            ax.scatter(pop_j[:, 0], pop_j[:, 1], s=24, color="red", edgecolors="none", alpha=alpha)

        # Return an empty tuple since we're redrawing everything
        return ()
    
    ax.set_xlim(x_bounds[0], x_bounds[1])
    ax.set_ylim(y_bounds[0], y_bounds[1])
    
    anim = animation.FuncAnimation(
        fig,
        update,
        frames=frames,
        interval=200,
        blit=False,  # must be False since we redraw each frame
    )
    
    writer = animation.PillowWriter(fps=5)
    anim.save(filename, writer=writer)
    plt.close(fig)
    print(f"Animation saved to {filename}")

### Exercise 1

Read [Cuckoo Search via Levy Flights](https://arxiv.org/pdf/1003.1594) with particular attention to Sections 2, 3, and 4. The primary focus (głównym punktem odniesienia) for this exercise is Figure 1, which outlines (naszkicowuje) the core pseudocode of the algorithm.

Your task is to implement the Cuckoo Search algorithm, using Figure 1 as your main reference. As the pseudocode is relatively high-level and lacks 
implementation details, you are encouraged to adopt a straightforward approach in your implementation.

**Action Item:** Document any ambiguities (niejasności or unclear aspects (nie jednoznaczności) you encounter (napotkasz) in the algorithm description or pseudocode.

> Note: If you find the implementation too challenging or feel stuck, you may proceed directly to Exercise 2.

In [ ]:
@dataclass
class CSResult:
    best_vector: np.ndarray
    best_value: float
    history: list[np.ndarray]  # History of populations for animation


class LevyFlight:
    def __init__(self, beta: float = 1.5):
        self.beta = beta

    def __call__(self, size: int) -> np.ndarray:
        sigma_u = (
            gamma(1 + self.beta)
            * np.sin(np.pi * self.beta / 2)
            / (gamma((1 + self.beta) / 2) * self.beta * 2 ** ((self.beta - 1) / 2))
        ) ** (1 / self.beta)
        sigma_v = 1
        u = np.random.normal(0, sigma_u, size)
        v = np.random.normal(0, sigma_v, size)
        step = u / np.abs(v) ** (1 / self.beta)
        return step


def cuckoo_search(
    func: Callable[[np.ndarray], float],
    bounds: list[tuple[float, float]] = BOUNDS,
    pop_size: int = 50,
    alpha: float = 1.0,
    beta: float = 1.5,
    p: float = 0.25,
    max_gen: int = 100,
) -> CSResult:
    """
    Implements the Cuckoo Search algorithm for global optimization.

    Parameters:
        func: Objective function to minimize. Takes a numpy array and returns a float.
        bounds: list of (min, max) pairs for each dimension.
        pop_size: Number of individuals in the population.
        alpha: Step-size scaling factor controlling the overall scale of the Lévy flights.
        beta: Exponent parameter for the Lévy distribution (typically in (1, 3]) that
            influences the heavy-tailed step-length distribution.
        p: Probability (in [0, 1]) that a host bird discovers an alien egg and abandons
            the corresponding nest—i.e., fraction of worse nests to be replaced each generation.
        max_gen: Maximum number of generations to evolve.
    """
   
    dimensions = len(bounds)
    lower_bounds = np.array([b[0] for b in bounds])
    upper_bounds = np.array([b[1] for b in bounds])

   
    #initialize population
    population = np.random.uniform(lower_bounds, upper_bounds, (pop_size, dimensions))
    fitness = np.array([func(ind) for ind in population])

    history = [population.copy()]

    levy = LevyFlight(beta=beta)

    for generation in range(max_gen):
        #generate new solutions 
        for i in range(pop_size):
            step = alpha * levy(dimensions)
            cuckoo = population[i] + step

            #boundaries 
            cuckoo = np.clip(cuckoo, lower_bounds, upper_bounds)
            cuckoo_fitness = func(cuckoo)

            # Select a random nest to potentially replace
            j = np.random.randint(pop_size)
            if cuckoo_fitness < fitness[j]:
                population[j] = cuckoo
                fitness[j] = cuckoo_fitness

        # Abandon a fraction (p) of worse nests and create new ones
        num_abandon = int(p * pop_size)
        worst_indices = np.argsort(fitness)[-num_abandon:]
        population[worst_indices] = np.random.uniform(lower_bounds, upper_bounds, (num_abandon, dimensions))
        fitness[worst_indices] = np.array([func(ind) for ind in population[worst_indices]])

        history.append(population.copy())

    best_idx = np.argmin(fitness)
    best_vector = population[best_idx]
    best_value = fitness[best_idx]

    return CSResult(best_vector, best_value, history)


### Test implemented CS

In [5]:
result = cuckoo_search(sphere, bounds=BOUNDS, pop_size=50)

### Experiments

Run CS on all three problems: Sphere, Rosenbrock and Rastrigin. For each problem:
- Visualize the population dynamics over time to illustrate how the search space is explored and exploited.

In [11]:
functions = {
    "sphere": sphere,
    "rosenbrock": rosenbrock,
    "rastrigin": rastrigin
}



for name, func in functions.items():
    result = cuckoo_search(func, bounds=BOUNDS, pop_size=50)
    animate_cs(func, result.history, bounds=BOUNDS, filename=f"{name}_cs.gif")
    print(f"{name.capitalize()} -> Best Value: {result.best_value}")

Animation saved to sphere_cs.gif
Sphere -> Best Value: 0.00021495630203771708
Animation saved to rosenbrock_cs.gif
Rosenbrock -> Best Value: 0.007167840622376968
Animation saved to rastrigin_cs.gif
Rastrigin -> Best Value: 0.17355044625112725


### Exercise 2

Read [An analysis of why cuckoo search does not bring any novel ideas to optimization](https://www.sciencedirect.com/science/article/pii/S0305054822000442). Focus particularly on Sections 2 and 3. Section 2.3 provides a detailed description of the implemented Cuckoo Search algorithm. Carefully analyze and compare it with your own, highlighting any differences (różnice) in assumptions (założeniach), parameter settings, or algorithmic structure.

My implementation of Cuckoo Search:    
- each individual is mutated using a Lévy flight    
- a new solution replaces a random one if it is better    
- some of the worst individuals are replaced by random solutions    
  
Algorithm version from the article:  
- each individual is also mutated using a Lévy flight, but the best individuals are selected    
- new solutions are generated by recombining two existing ones (inspired by Differential Evolution)    

### Exercise 3

Read the Introduction of “An analysis of why cuckoo search does not bring any novel ideas to optimization.” Identify and outline three  
criteria proposed by the authors for evaluating the underlying metaphor (leżącej u podstaw metafory) of the algorithm. Critically reflect   
(zastanów się) on these criteria, do you find them appropriate and sufficient (wystarczające)? Can you suggest any additional criteria or alternative   
perspectives that might enrich (wzbogacić) the evaluation?  

In the introduction to the article, the authors propose three criteria to evaluate the metaphor:
- Usefulness. Does the metaphor bring useful concepts to solve
 optimization problems?
-  Novelty. Were the concepts brought by the metaphor new in
 the field of stochastic optimization at the time when they were
 proposed?
-  Sound motivation. Is there a sound motivation to use the
 metaphor?

I think these criteria are appropriate, but I would suggest an additional one to enrich the evaluation.  
A possible fourth criterion could be computational effectiveness – that is, whether the metaphor leads   
to algorithms that are actually efficient in practice.  

### Exercise 4

Read Section 4 of “An analysis of why cuckoo search does not bring any novel ideas to optimization.” Explain the main criticisms they raise against the Cuckoo Search algorithm. What fundamental issues do they identify, and how do these undermine (podważają) the algorithm's novelty?


No new ideas - Cuckoo Search reuses old techniques from evolution strategies like (μ + λ)-ES. It doesn’t bring anything truly new.  
Uses parts from other algorithms - The algorithm borrows recombination from Differential Evolution and uses Lévy flights for mutation — both of which were already known.  
Wrong order of operations - CS applies recombination at the end of each loop, not at the start like in standard evolutionary strategies. This wastes time and breaks the typical flow.



### Exercise 5

Analyze Figures 5 and 8 from [Large-scale Benchmarking of Metaphor-based Optimization Heuristics](https://arxiv.org/pdf/2402.09800). In these figures, Cuckoo Search is denoted as CS. Evaluate its performance relative to the CMA-ES variant (bipop) and Differential Evolution (DE). How does CS compare to these well-established algorithms in terms of optimization performance? Furthermore, critically consider whether performance alone is a sufficient criterion for evaluating optimization algorithms. What other factors should be taken into account?

In Figures 5 and 8 of the paper Large-scale Benchmarking of Metaphor-based Optimization Heuristics, the Cuckoo Search (CS)   
algorithm performs worse than well-known algorithms like CMA-ES (bipop) and Differential Evolution (DE).  

CS sometimes gives good results, especially with small budgets and low dimensions, but usually loses to DE and CMA-ES.  

The authors also say that performance is not the only thing that matters when evaluating algorithms. We should also think about  
Practical usefulness, whether it fits the theory or metaphor it is based on.